# Analyse touch chart
in this notebook, I check that the collected touch charts are correct. this means, the touch charts need to be lying onto the object surface once transdlated from workframe to worldframe.

In [1]:
import numpy as np
import results
import os
import plotly.graph_objects as go
import trimesh
from utils import utils_mesh
import pybullet as pb
from utils import utils_raycasting, utils_mesh
from data import objects


pybullet build time: Aug 27 2022 18:18:13


Load the dictionary:

In [2]:
touch_charts_dict_path = os.path.join(os.path.dirname(results.__file__), 'touch_charts_gt.npy')
touch_charts_dict = np.load(touch_charts_dict_path, allow_pickle=True).item()

In [3]:
touch_charts_dict.keys()

dict_keys(['verts', 'tactile_imgs', 'pointclouds', 'rot_M_wrld_list', 'pos_wrld_list', 'pos_wrk_list', 'obj_index', 'initial_pos'])

Let's visualise the pointclouds sampled on the object:

In [4]:
objs_dict_path = os.path.join(os.path.dirname(results.__file__), 'objs_dict.npy')
objs_dict = np.load(objs_dict_path, allow_pickle=True).item()

In [5]:
objs_dict.keys()

dict_keys(['3398', '3763', '102505', '100435', '100466', '100468', '102763', '3618', '3616', '102431', '102408', '3635', '3517', '103062', '3519', '103030', '103064', '3520', '103016', '100439', '3596', '100465', '102506', '100438', '100749', '3678', '3614', '3625', '3822', '102417', '3615', '102432', '3655', '102434', '102523', '101352', '3571', '101362'])

In [6]:
# Indexes of the object in touch_charts_dict
obj_indexes = np.unique(touch_charts_dict['obj_index'])

# Let's focus on the first object:
obj_index = obj_indexes[1]

# Load scaled and rotated pointcloud of the object
obj_path = os.path.join(os.path.dirname(objects.__file__), obj_index) 
mesh = utils_mesh.urdf_to_mesh(obj_path)
obj_pointcloud = utils_mesh.mesh_to_pointcloud(mesh, n_samples = 5000)
obj_pointcloud = utils_mesh.scale_pointcloud(obj_pointcloud, scale=0.1)
obj_pointcloud = utils_mesh.rotate_pointcloud(obj_pointcloud)

tc_indexes = np.where(touch_charts_dict['obj_index'] == obj_index)[0]

tc_pointclouds_wrk = touch_charts_dict['pointclouds'][tc_indexes]
pos_wrld_list = touch_charts_dict['pos_wrld_list'][tc_indexes]
rot_M_wrld_list = touch_charts_dict['rot_M_wrld_list'][tc_indexes]
rot_M_wrld_list = touch_charts_dict['rot_M_wrld_list'][tc_indexes]
initial_pos = touch_charts_dict['initial_pos'][tc_indexes][0]

tc_pointclouds_wrld = utils_mesh.translate_rotate_mesh(pos_wrld_list, rot_M_wrld_list, tc_pointclouds_wrk, initial_pos)
print(tc_pointclouds_wrld.shape)
tc_x = tc_pointclouds_wrld[:, :, 0].ravel() 
tc_y = tc_pointclouds_wrld[:, :, 1].ravel()
tc_z = tc_pointclouds_wrld[:, :, 2].ravel() 

fig = go.Figure(
    [
        go.Scatter3d(x=obj_pointcloud[:, 0], y=obj_pointcloud[:, 1],z=obj_pointcloud[:, 2], 
                     mode='markers', marker=dict(size=1)),
        go.Scatter3d(x=tc_x, y=tc_y, z=tc_z, 
                     mode='markers', marker=dict(size=1))
    ]
)
fig.show()    
    

IndexError: index 1 is out of bounds for axis 0 with size 1

Count the number of points in the point cloud: